In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import logging
from typing import Optional

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from dotenv import load_dotenv
import os
import time

def click_social_button(driver, icon_class, network_name):
    """Helper function to click social media buttons"""
    try:
        print(f"Looking for {network_name} button...")
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                f"//div[contains(@class, 'flex-grow-0')]//i[contains(@class, 'fa-{icon_class}')]/ancestor::div[contains(@class, 'cursor-pointer')]"
            ))
        )
        
        if "opacity-50" in button.get_attribute("class"):
            print(f"{network_name} button appears disabled. Skipping click.")
            return
        
        print(f"Scrolling into view for {network_name} button...")
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        
        print(f"Hovering over and clicking {network_name} button...")
        action = ActionChains(driver)
        action.move_to_element(button).click(button).perform()
        time.sleep(1)
        
    except Exception as e:
        print(f"Error clicking {network_name} button: {str(e)}")

def test_metricool_login():
    load_dotenv()
    email = os.getenv('METRICOOL_EMAIL')
    password = os.getenv('METRICOOL_PASSWORD')
    
    if not email or not password:
        print("Error: Please set METRICOOL_EMAIL and METRICOOL_PASSWORD in your .env file")
        return False
    
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        print("Navigating to Metricool homepage...")
        driver.get("https://metricool.com/")
        
        print("Looking for login button...")
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "cta-blanco-login"))
        )
        login_button.click()
        
        time.sleep(3)
        
        print(f"Current URL: {driver.current_url}")
        
        email_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_username"))
        )
        email_input.clear()
        email_input.send_keys(email)
        
        password_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_password"))
        )
        password_input.clear()
        password_input.send_keys(password)
        
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "loginFormSubmit"))
        )
        submit_button.click()
        
        planner_button = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/planner'] .fa-calendar-days"))
        )
        planner_button.click()
        
        time.sleep(10)
        
        create_post_button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.v-btn.primary .fa-plus"))
        )
        create_post_button.click()
        
        time.sleep(7)
        
        content_editor = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.editor-box[contenteditable='true']"))
        )
        driver.execute_script("arguments[0].innerHTML = 'hola'", content_editor)
        time.sleep(2)
        
        social_networks = [
            ('x-twitter', 'Twitter'),
            ('facebook', 'Facebook'),
            ('linkedin', 'LinkedIn')
        ]
        
        for icon_class, network_name in social_networks:
            try:
                click_social_button(driver, icon_class, network_name)
            except Exception as e:
                print(f"Error with {network_name}: {str(e)}")
                continue
        
        print("All social media buttons processed! Keeping browser open...")
        
        while True:
            time.sleep(1)
            
    except TimeoutException as e:
        print(f"Timeout error: Element not found - {str(e)}")
        return False
    except KeyboardInterrupt:
        print("\nScript interrupted by user. Closing browser...")
        driver.quit()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False

if __name__ == "__main__":
    test_metricool_login()


Navigating to Metricool homepage...
Looking for login button...
Current URL: https://app.metricool.com/
Looking for Twitter button...
Scrolling into view for Twitter button...
Hovering over and clicking Twitter button...
Looking for Facebook button...
Scrolling into view for Facebook button...
Hovering over and clicking Facebook button...
Looking for LinkedIn button...
Scrolling into view for LinkedIn button...
Hovering over and clicking LinkedIn button...
All social media buttons processed! Keeping browser open...

Script interrupted by user. Closing browser...


In [ ]:

class MetricoolPoster:
    def __init__(self, email: str, password: str):
        """
        Initialize MetricoolPoster with login credentials.
        
        Args:
            email (str): Metricool login email
            password (str): Metricool login password
        """
        self.email = email
        self.password = password
        self.driver = None
        self.logger = self._setup_logging()
        
    def _setup_logging(self) -> logging.Logger:
        """Configure logging for the Metricool automation."""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('metricool_automation.log'),
                logging.StreamHandler()
            ]
        )
        return logging.getLogger(__name__)
    
    def _setup_driver(self):
        """Set up Chrome WebDriver with appropriate options."""
        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')
        # Uncomment below line for headless mode
        # options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=options)
        
    def _wait_and_find_element(self, by: By, value: str, timeout: int = 10) -> Optional[webdriver.remote.webelement.WebElement]:
        """
        Wait for and find an element on the page.
        
        Args:
            by: Selenium By method
            value: Element identifier
            timeout: Maximum time to wait in seconds
            
        Returns:
            WebElement if found, None otherwise
        """
        try:
            element = WebDriverWait(self.driver, timeout).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except TimeoutException:
            self.logger.error(f"Element not found: {value}")
            return None
            
    def login(self) -> bool:
        """
        Log in to Metricool.
        
        Returns:
            bool: True if login successful, False otherwise
        """
        try:
            self.logger.info("Starting login process")
            self.driver.get("https://app.metricool.com/login")
            
            # Wait for and fill email
            email_input = self._wait_and_find_element(By.NAME, "email")
            if not email_input:
                return False
            email_input.send_keys(self.email)
            
            # Wait for and fill password
            password_input = self._wait_and_find_element(By.NAME, "password")
            if not password_input:
                return False
            password_input.send_keys(self.password)
            
            # Click login button
            login_button = self._wait_and_find_element(By.CSS_SELECTOR, "button[type='submit']")
            if not login_button:
                return False
            login_button.click()
            
            # Wait for dashboard to load
            dashboard = self._wait_and_find_element(By.CLASS_NAME, "dashboard")
            return dashboard is not None
            
        except Exception as e:
            self.logger.error(f"Login failed: {str(e)}")
            return False
            
    def post_tweet(self, tweet_text: str) -> bool:
        """
        Post a tweet to Metricool.
        
        Args:
            tweet_text (str): The tweet content to post
            
        Returns:
            bool: True if posting successful, False otherwise
        """
        try:
            self.logger.info("Starting tweet posting process")
            
            # Navigate to post creation
            post_button = self._wait_and_find_element(By.CSS_SELECTOR, "button[data-testid='create-post-button']")
            if not post_button:
                return False
            post_button.click()
            
            # Wait for and fill tweet text
            tweet_input = self._wait_and_find_element(By.CSS_SELECTOR, "div[contenteditable='true']")
            if not tweet_input:
                return False
            tweet_input.send_keys(tweet_text)
            
            # Select Twitter platform (if needed)
            twitter_checkbox = self._wait_and_find_element(By.CSS_SELECTOR, "input[type='checkbox'][value='twitter']")
            if twitter_checkbox and not twitter_checkbox.is_selected():
                twitter_checkbox.click()
            
            # Click post button
            submit_button = self._wait_and_find_element(By.CSS_SELECTOR, "button[type='submit']")
            if not submit_button:
                return False
            submit_button.click()
            
            # Wait for confirmation
            time.sleep(3)
            return True
            
        except Exception as e:
            self.logger.error(f"Posting tweet failed: {str(e)}")
            return False
            
    def close(self):
        """Close the browser and clean up."""
        if self.driver:
            self.driver.quit()
            
    def __enter__(self):
        """Context manager entry."""
        self._setup_driver()
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit."""
        self.close()

def post_to_metricool(email: str, password: str, tweet: str) -> bool:
    """
    Main function to post a tweet to Metricool.
    
    Args:
        email (str): Metricool login email
        password (str): Metricool login password
        tweet (str): Tweet content to post
        
    Returns:
        bool: True if posting successful, False otherwise
    """
    with MetricoolPoster(email, password) as poster:
        if poster.login():
            return poster.post_tweet(tweet)
        return False

# Example usage
if __name__ == "__main__":
    from dotenv import load_dotenv
    import os
    
    load_dotenv()
    
    METRICOOL_EMAIL = os.getenv('METRICOOL_EMAIL')
    METRICOOL_PASSWORD = os.getenv('METRICOOL_PASSWORD')
    
    test_tweet = "¡Hola! Este es un tweet de prueba 🚀 #ITAM"
    
    success = post_to_metricool(METRICOOL_EMAIL, METRICOOL_PASSWORD, test_tweet)
    print(f"Tweet posted successfully: {success}")